In [1]:
%matplotlib inline
#some basic imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

#TODO: add your imports here
from elasticsearch import Elasticsearch

#seting up some styles
import seaborn as sns
sns.set_context("paper")
sns.set_style("whitegrid")

#avoid Font-problem in plots for ACM/IEEE papers
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

experiment_name = "DITAS-Performance-Benchmark"

es = Elasticsearch(host="153.92.30.56",
                   http_auth=('admin', 'ditasmetrics'),
                   scheme="http",
                   port=50035)

In [10]:
#getting data
from load import load,load_monitoring
use_cache=False

data_cache_file="{}_{}.data.csv".format(experiment_name,datetime.now().strftime("%d-%m-%Y"))
data = load(load_from_cache=use_cache,store_cache_file=True,data_cache_file=data_cache_file)

In [11]:
experiment_dates =list(map(lambda x:pd.Timestamp(x),data['runDate'].unique()))

mon__cache_file="{}_{}.mon.csv".format(experiment_name,datetime.now().strftime("%d-%m-%Y"))
monData = load_monitoring(load_from_cache=use_cache,store_cache_file=True,monitoring_cache_file=mon__cache_file,es=es,experiment_dates=experiment_dates)

In [17]:
data.groupby(["experiment","method"])["response-time"].mean()

experiment  method                 
B0          GetSimplifiedDiagnostic    18.674000
            GetStreamingData           30.000462
B1          GetStreamingData           30.000866
B4A         GetStreamingData           30.000427
B4B         GetStreamingData           30.000426
B9          GetStreamingData           30.000307
Name: response-time, dtype: float64

## your experiment

In [ ]:
#usefull commands
data[data['startTime'] > pd.Timestamp(2019,12,4,13)]
#save a table
with open('images/{tabel}.tex', 'w') as tf:
     tf.write(data.to_latex(escape=False))

#save a plot
plt.savefig("images/{filename}.pdf", bbox_inches="tight")